In [23]:
import csv
from functools import reduce

BLUE = "\033[1;34m"
RESET = "\033[0;0m"
BOLD = "\033[;1m"

def escolhas(x,y):
  try:
    y[int(x)-1]()
  except (IndexError,ValueError):
    print("Escolha inválida. Por favor, tente novamente.")
    x = input("")
    escolhas(x,y)

def inicio():
  print(BLUE + "Olá, bem-vindo ao Adafy, o seu streaming de música favorito." + RESET)
  print("Digite 1 para login de usuário")
  print("Digite 2 para login de administrador")
  print("Digite 3 para sair")
  escolha = input("")
  opcoes = (loginuser, loginadm, saida)
  escolhas(escolha,opcoes)

def loginuser():
  print("Você entrou no modo usuário, por favor, escolha uma das opções abaixo:")
  print("Digite 1 para buscar playlist")
  print("Digite 2 para criar playlist")
  print("Digite 3 para sair")
  escolha = input("")
  opcoes = (buscarplaylist, novaplaylist, saida)
  escolhas(escolha,opcoes)

def loginadm():
  print("Você entrou no modo adm, por favor, escolha uma das opções abaixo:")
  print("Digite 1 para registrar artista")
  print("Digite 2 para registrar album")
  print("Digite 3 para sair")
  escolha = input("")
  opcoes = (novoartista, novoalbum, saida)
  escolhas(escolha,opcoes)

def novoartista():
  cantor = input("Digite o novo artista que deseja cadastrar: ")
  with open('Artistas.csv', 'r+') as arquivo:
    database = list(csv.reader(arquivo, delimiter = ',', lineterminator="\n"))
    cantores = [x[0] for x in database]
    if cantor in cantores:
      print(BOLD + "Artista já cadastrado no Adafy." + RESET)
    else:
      pais = input("Digite o país do artista: ")
      update = [cantor,pais,""]
      writer_object = csv.writer(arquivo)
      writer_object.writerow(update)
      print(BOLD + "Artista cadastrado com sucesso." + RESET)
  loginadm()

def novoalbum():
  cantor = input("Digite o artista do album: ")
  with open('Artistas.csv','r') as arquivo:
    database_artistas = list(csv.reader(arquivo, delimiter = ',', lineterminator="\n"))
  cantores = [x[0] for x in database_artistas]
  paises = [x[1] for x in database_artistas]
  if cantor not in cantores:
    print(BOLD + "Artista ainda não registrado no Adafy. Efetue seu cadastro antes." + RESET)
    loginadm()
  else:
    pais = str([x[1] for x in zip(cantores,paises) if x[0] == cantor][0])
    cadastroalbum(cantor,pais)

def infosiniciais():
  with open('Albuns.csv', 'r') as arquivo1:
    database_albuns = list(csv.reader(arquivo1, delimiter = ',', lineterminator="\n"))
  with open('Artistas.csv', 'r') as arquivo2:
    database_artistas = list(csv.reader(arquivo2, delimiter = ',', lineterminator="\n"))
  albuns = [x[0] for x in database_albuns]
  cantores = [x[0] for x in database_artistas]
  paises = [x[1] for x in database_artistas]
  albuns_antigos = [x[2] for x in database_artistas]
  antigo = [x for x in zip(cantores,paises,albuns_antigos)]
  return albuns, antigo, database_artistas 

def cadastroalbum(cantordesejado,paisdesejado):
  album = input("Digite o novo álbum que deseja cadastrar: ")
  albuns, antigo, database_artistas = infosiniciais()
  if album in albuns:
    print(BOLD + "Álbum já cadastrado no Adafy." + RESET)
  else:
    ano = input("Digite o ano de lançamento do album: ")
    try:
      qte_musicas = int(input("Quantas músicas há no album? "))
      musicas = list(map(lambda i: input(f"Digite a música n° {i}: "), range(1,qte_musicas + 1)))
      musicas = reduce(lambda x,y: x+";"+y,musicas)
      update_album = [album,cantordesejado,paisdesejado,ano,musicas]
      with open('Albuns.csv', 'a') as arquivo1:
        writer_object2 = csv.writer(arquivo1)
        writer_object2.writerow(update_album)
      for _ in antigo:
        if _[0] == cantordesejado:
          albuns_upd = album if _[2] == "" else _[2] + ";" + album
          update_cantor = [cantordesejado,paisdesejado,albuns_upd]
          database_artistas.remove(list(_))
          database_artistas.append(update_cantor)
      with open('Artistas.csv', 'w') as arquivo3:
          writer = csv.writer(arquivo3)
          writer.writerows(database_artistas)
          print(BOLD + "Álbum cadastrado com sucesso." + RESET)
    except (ValueError,TypeError):
      print(BOLD + "Não foi possível cadastrar o álbum, tente novamente" + RESET)
  loginadm()

def buscarplaylist():
  print("Digite 1 para buscar playlist por música")
  print("Digite 2 para buscar playlist por artista")
  print("Digite 3 para buscar playlist por nome")
  escolha = input("")
  opcoes = (buscarplaylist_musica, buscarplaylist_artista, buscarplaylist_nome)
  escolhas(escolha,opcoes)  

def buscarplaylist_musica():
  musicadesejada = input("Digite a música desejada: ")
  busca_padrao(musicadesejada)

def buscarplaylist_artista():
  artistadesejado = input("Digite o artista desejado: ")
  busca_padrao(artistadesejado)

def busca_padrao(inputdesejado):
  with open("Playlists.csv",'r') as arquivo:
    database = list(csv.reader(arquivo, delimiter = ',', lineterminator="\n"))
  cabecalho, *corpo = database
  dic = dict(corpo)
  play = [x for x in dic if inputdesejado in dic[x]]
  if play == []:
    print(BOLD + f"Não há playlists criadas com {inputdesejado}." + RESET)
  else:
    print(BOLD + f"As playlists que incluem {inputdesejado} são: {', '.join(play)}." + RESET)
  loginuser()

def buscarplaylist_nome():
  with open('Playlists.csv', 'r') as arquivo:
      database_playlist = list(csv.reader(arquivo, delimiter = ',', lineterminator="\n"))
  cabecalho, *playlists = database_playlist
  dic_playlists = dict(playlists)
  nomes_playlists = reduce(lambda x,y: x+", "+y,list(dic_playlists.keys()))
  print(BOLD + f"Suas playlists são: {nomes_playlists}." + RESET)
  buscar = input("Digite o nome da playlist que deseja visualizar as músicas: ")
  if buscar in list(dic_playlists.keys()):
    musicas = dic_playlists[buscar].replace(";",", ")
    print(BOLD + f"As músicas da playlist {buscar} são: {musicas}." + RESET)
  else:
    print(BOLD + "Playlist não localizada." + RESET)
  loginuser()

def novaplaylist():
  with open('Playlists.csv', 'r') as arquivo:
    database_playlist = list(csv.reader(arquivo, delimiter = ',', lineterminator="\n"))
  cabecalho, *playlists = database_playlist
  dic_playlists = dict(playlists)
  cadastro = input("Digite o nome para sua nova playlist: ")
  if cadastro in list(dic_playlists.keys()):
    print(BOLD + "Nome já atribuído a outra playlist." + RESET)
    loginuser()
  else:
    musicas = []
    while True:
      print("Digite 1 para cadastrar música.")
      print("Digite 2 para finalizar a nova playlist.")
      continuar = int(input(""))
      if continuar == 2:
        break
      elif continuar != 1:
        print("Opção inválida.")
      else:
        musicacadastro = cadastroplaylist_passo1()
        musicas.append(musicacadastro)
    cadastroplaylist_passo3(cadastro,musicas)

def cadastroplaylist_passo1():
  with open('Artistas.csv','r') as arquivo:
    database_artistas = list(csv.reader(arquivo, delimiter = ',', lineterminator="\n"))
  cabecalho, *artistas = database_artistas
  chaves = [x[0] for x in artistas]
  valores = [(x[1] + ", " + x[2]) for x in artistas]
  dic_artistas = dict(zip(chaves,valores))
  print(BOLD + f"Os artistas disponíveis no Adafy são: {', '.join(chaves)}" + RESET)
  artista = input("Digite o artista desejado: ")
  try:
    albuns = ((dic_artistas[artista].replace(" ","").split(","))[1]).split(";")
    print(BOLD + f"Os álbuns disponíveis de {artista} são: {', '.join(albuns)}" + RESET)
    album =  input("Digite o album desejado: ")
    if album in albuns:
      musicacadastro = cadastroplaylist_passo2(album,artista)
    else:
      print(BOLD + "Album não cadastrado" + RESET)
      musicacadastro = cadastroplaylist_passo1()
  except KeyError:
    print(BOLD + "Artista não cadastrado" + BOLD)
    musicacadastro = cadastroplaylist_passo1()
  return musicacadastro

def cadastroplaylist_passo2(albumescolhido,artistaescolhido):
  try:
    with open('Albuns.csv','r') as arquivo:
      database_albuns = list(csv.reader(arquivo, delimiter = ',', lineterminator="\n"))
    cabecalho, *albuns = database_albuns
    chaves = [x[0] for x in albuns]
    valores = [x[4] for x in albuns]
    dic_albuns = dict(zip(chaves,valores))
    print(BOLD + f"As músicas disponíveis do {albumescolhido} são: {dic_albuns[albumescolhido].replace(';',', ')}" + RESET)
    musica = input("Digite a música desejada: ")
    if musica in dic_albuns[albumescolhido].split(";"):
      musicacadastro = musica + "(" + artistaescolhido + " - " + albumescolhido + ")"
    else:
      print(BOLD + "Música não disponível no Adafy" + RESET)
      musicacadastro = cadastroplaylist_passo1()
  except KeyError:
    print("Erro no cadastro, tente novamente.")
    musicacadastro = cadastroplaylist_passo1()
  return musicacadastro

def cadastroplaylist_passo3(nomeescolhido,musicasescolhidas):
  if musicasescolhidas == []:
    print(BOLD + "Não é possível cadastrar playlist vazia." + RESET)
  else:
    update = [nomeescolhido,";".join(musicasescolhidas)]
    with open('Playlists.csv','a') as arquivo:
      writer_object = csv.writer(arquivo)
      writer_object.writerow(update)  
    print(BOLD + "Playlist criada com sucesso." + RESET)
  loginuser()

def saida():
  print(BOLD + BLUE + "Esperamos você em breve novamente!")


inicio()

Olá, bem-vindo ao Adafy, o seu streaming de música favorito.
Digite 1 para login de usuário
Digite 2 para login de administrador
Digite 3 para sair
1
Você entrou no modo usuário, por favor, escolha uma das opções abaixo:
Digite 1 para buscar playlist
Digite 2 para criar playlist
Digite 3 para sair
1
Digite 1 para buscar playlist por música
Digite 2 para buscar playlist por artista
Digite 3 para buscar playlist por nome
1
Digite a música desejada: Bang
As playlists que incluem Bang são: Diversao.
Você entrou no modo usuário, por favor, escolha uma das opções abaixo:
Digite 1 para buscar playlist
Digite 2 para criar playlist
Digite 3 para sair
1
Digite 1 para buscar playlist por música
Digite 2 para buscar playlist por artista
Digite 3 para buscar playlist por nome
2
Digite o artista desejado: Beyonce
As playlists que incluem Beyonce são: Diversao, Relaxar.
Você entrou no modo usuário, por favor, escolha uma das opções abaixo:
Digite 1 para buscar playlist
Digite 2 para criar playlist
Di